In [1]:
# %pip install pytorch_lightning
# %pip install transformers
# %pip install torchmetrics
# %pip install soundfile
# %pip install librosa
# %pip install ipywidgets

In [2]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
# import math
import pytorch_lightning as pl
# import torchaudio
import torchmetrics
from torch.optim.lr_scheduler import ReduceLROnPlateau

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(


In [3]:
torch.cuda.is_available()

True

In [4]:
from local_dataset import AudioEmotionsDataset 
# import TQDMProgressBar
from pytorch_lightning.callbacks import TQDMProgressBar

In [12]:
BATCH_SIZE = 32
dataset = AudioEmotionsDataset("/home/ubuntu/siavava-west-1/test/data/audio-emotions", batch_size=BATCH_SIZE, max_size=200, feature_type="mfcc")

train = dataset.train_dataloader
test = dataset.test_dataloader


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# # visualize one fetures 2D array using matplotlib
# import matplotlib.pyplot as plt
# import numpy as np
# import librosa.display

# # %pip install resampy

# def extract_mfcc_features(file: str):
#     """
#         Loads audio from a given file path and extracts features using the MFCC algorithm.
#     """
    
#     # NOTE: resample to 16kHz
#     waveform, sample_rate = librosa.load(file, res_type='kaiser_fast')
#     # mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    
#     #? extract features
#     features = librosa.feature.mfcc(y=waveform, sr=sample_rate, n_mfcc=40)

#     print(f"{features.shape = }")

#     features = torch.mean(torch.tensor(features), axis=1)

#     print(f"{features.shape = }")

#     features = torch.tensor(features)

#     # m = nn.ZeroPad2d((0, 128 - features.shape[1]))
    
#     # features = m(features)[:, :128]

#     # # trim any excess to (128, 128)
#     # features = features[:, :128]
#     # features = torch.mean(features, axis=1)

#     # print(f"{features = }")

#     # print(f"{features.shape = }")
#     # features = features.unsqueeze(1)
#     # print(f"{features.shape = }")
    
#     return waveform, features

# files = [
#       "/home/ubuntu/siavava-west-1/test/data/audio-emotions/Angry/03-01-05-01-01-01-01.wav"
#     , "/home/ubuntu/siavava-west-1/test/data/audio-emotions/Disgusted/03-01-07-01-01-01-02.wav"
#     , "/home/ubuntu/siavava-west-1/test/data/audio-emotions/Fearful/03-01-06-01-01-01-01.wav"
#     , "/home/ubuntu/siavava-west-1/test/data/audio-emotions/Happy/03-01-03-01-01-01-01.wav"
#     , "/home/ubuntu/siavava-west-1/test/data/audio-emotions/Neutral/03-01-01-01-01-01-02.wav"
#     , "/home/ubuntu/siavava-west-1/test/data/audio-emotions/Sad/03-01-04-01-01-01-01.wav"
#     , "/home/ubuntu/siavava-west-1/test/data/audio-emotions/Surprised/03-01-08-01-01-01-01.wav"
# ]

# # get one batch of data
# # for file in files:
# #     waveform, features = extract_mfcc_features(file)
# #     print(f"{features.shape = }")
# #     plt.figure(figsize=(10, 4))
# #     librosa.display.specshow(features.numpy(), x_axis='time')
# #     plt.colorbar()
# #     plt.title(f"MFCC - {file.split('/')[2]}")
# #     plt.tight_layout()
# #     plt.show()
# # _, x = extract_mfcc_features("data/audio-emotions/Angry/03-01-05-01-01-01-01.wav")
# # # x, y = batch
# # print(x.shape)
# # plt.figure(figsize=(10, 4))
# # librosa.display.specshow(x[:10].numpy(), x_axis='time')
# # plt.colorbar()
# # plt.title('MFCC')
# # plt.tight_layout()
# # plt.show()
# # break

In [14]:
m1 = nn.Conv1d(in_channels=1, kernel_size=40, out_channels=7)

m2 = nn.ReLU()

m3 = nn.Dropout(p=0.2)

m4 = nn.Flatten(start_dim=1, end_dim=2)

m5 = nn.Linear(2304, 7)

m6 = nn.Softmax(dim=1)


# m2 = nn.Linear(64, 7)

# m2 = nn.Linear(7, 7)


# m4 = nn.Linear(7, 7)

m5 = nn.Softmax(dim=1)

# m6 = nn.Softmax(dim=1)

# m1 = nn.Conv1d(in_channels=1, out_channels=40, padding="same")

# m2 = nn.ReLU()

# m3 = nn.Dropout(p=0.2)

# m4 = nn.Flatten()

# m5 = nn.Linear(5, 7)

# m6 = nn.Softmax(dim=1)

# weighted average
# w = nn.Parameter(torch.randn(1024, 1))

for batch in train:
    X, y = batch
    # print(f"{X}")
    # print(X[0], "\n", X.shape)
    # X = X.unsqueeze(1)
    # print(X[0], "\n", X.shape)

    print(f"{X.shape = }")

    X = X.view(X.shape[0], 1, X.shape[1])

    for i, layer in enumerate([m1, m2, m3, m4, m5, m6]):
        X = layer(X)
        print(i, X.shape)
    
    break

X.shape = torch.Size([32, 40])
0 torch.Size([32, 7, 1])
1 torch.Size([32, 7, 1])
2 torch.Size([32, 7, 1])
3 torch.Size([32, 7])
4 torch.Size([32, 7])
5 torch.Size([32, 7])


In [19]:
class Reshape(nn.Module):
    def __init__(self):
        super(Reshape, self).__init__()
        # self.shape = shape

    def forward(self, x):
        # return x.view((x.shape[0], *self.shape))
        return x.view(x.shape[0], 1, x.shape[1])
    
    def __call__(self, x):
        return self.forward(x)
    
class Squeeze(nn.Module):
    def __init__(self):
        super(Squeeze, self).__init__()

    def forward(self, x):
        return x.squeeze()
    
    def __call__(self, x):
        return self.forward(x)

class Print(nn.Module):
    def __init__(self, name: int=0):
        super(Print, self).__init__()
        self.name = name

    def forward(self, x):
        # print(f"{self.name:2d}: {x.shape}")
        return x
    
    def __call__(self, x):
        return self.forward(x)
    


class SpeechEmotionRecognitionModel(pl.LightningModule):
    def __init__(self, input_size, num_classes, dim_feedforward=2048, dim_model=1024, nhead=8, num_encoder_layers=6, num_decoder_layers=6, lr=0.5, dropout=0.1):
        super(SpeechEmotionRecognitionModel, self).__init__()
        self.lr = lr

        # encoder_layers = nn.TransformerEncoderLayer(dim_model, nhead, dim_feedforward, dropout)
        # self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        # self.encoder = nn.Linear(input_size, dim_model)
        # self.decoder = nn.Linear(dim_model, num_classes)

        # self.layers = nn.Sequential(
        #     # nn.Conv1d(in_channels=13, kernel_size=11, out_channels=7),
        #     nn.Linear(input_size, 64),
        #     nn.Linear(64, 7),
        #     nn.Dropout(p=0.1),
        #     nn.Linear(7, 7),
        #     nn.ReLU(),
        #     nn.Linear(7, 7),
        #     # nn.Softmax(dim=1),
        #     # nn.Sigmoid(),
        #     nn.Softmax(dim=1)
        # )

        self.layers = nn.Sequential(
            Reshape(),
            nn.Conv1d(in_channels=1, kernel_size=40, out_channels=7),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Flatten(start_dim=1, end_dim=2),
            nn.Linear(7, 7),
            # nn.Softmax(dim=1),
            nn.Sigmoid()
        )
        

        # initialize the metrics
        self.loss_function = nn.CrossEntropyLoss()
        self.precision = torchmetrics.Precision(task='multiclass', num_classes=num_classes, average="macro")
        self.recall = torchmetrics.Recall(task='multiclass', num_classes=num_classes, average="macro")
        self.F1 = torchmetrics.F1Score(task='multiclass', num_classes=num_classes, average="macro")

    def forward(self, src):
        output = self.layers(src)
        return output

    def training_step(self, batch, batch_idx):
        src, tgt = batch
        
        # src = src.unsqueeze(1)
        # print(f"{src.shape = }")
        output = self(src)

        # print(f"{output.shape = }")
        # print(f"{tgt.shape = }")
        # loss = self.loss_function(output, tgt)
        loss = self.loss_function(output, torch.argmax(tgt, axis=1))
        self.log('cross entropy loss_step', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        src, tgt = batch
        output = self(src)
        loss = self.loss_function(output, torch.argmax(tgt, axis=1))
        # self.log('cross entropy loss', loss, on_epoch=True, prog_bar=True)
        # print(f"LOSS: {loss}")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = {
            'scheduler': ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True),
            'monitor': 'cross entropy loss_step',  # Name of the metric to monitor
            'interval': 'epoch',
            'frequency': 1,
        }
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

    # function for evaluating the quality of output and target
    def evaluation(self, output, target, loss):
        # Calculate and log metrics
        # output = output.squeeze()

        precision = self.precision(output, target)
        recall = self.recall(output, target)
        f1 = self.F1(output, target)
        # print('precision:{}'.format(self.precision(output, target)))
        # self.log('recall:{}'.format(self.recall(output, target)))
        # self.log('f1:{}'.format(self.F1(output, target)))

        print(f"CE:        {loss}")
        print(f"PRECISION: {precision}")
        print(f"RECALL:    {recall}")
        print(f"F1:        {f1}")

        # self.log('precision', precision)
        # self.log('recall', recall)
        # self.log('f1', f1)

In [20]:
model = SpeechEmotionRecognitionModel(input_size=dataset.feature_count, num_classes=dataset.class_count)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
# callbacks=[TQDMProgressBar(refresh_rate=10)]
trainer = pl.Trainer(default_root_dir='checkpoints', accelerator="auto", max_epochs=50, min_epochs=2, log_every_n_steps=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [29]:
for batch in test:
    X, y = batch
    X = X.cuda(0)
    y = y.cuda(0)
    print(f"X: {X.device}")
    print(f"y: {y.device}")
    print(f"model: {model.device}")

    model = model.cuda(0)
    output = model(X)
    # print out put device
    print(f"output: {X.device}")
    model.evaluation(output, y, model.loss_function(output, y))

    # print(f"\n\nACTUAL")
    # print(y)

    # print the predictions
    # print(f"\n\nPredictions: {torch.argmax(output, dim=1)}")
    # # print sums of predictions
    # print(f"Predictions sum: {torch.sum(output, dim=1)}")
    # print(output)

    for i in range(32):
        print(f"{torch.argmax(output[i]):2d} | {torch.argmax(y[i]):2d} >>> {torch.max(output[i]):.2f}")
    break

X: cuda:0
y: cuda:0
model: cuda:0
output: cuda:0
CE:        1.814352035522461
PRECISION: 0.5845528244972229
RECALL:    0.6354166269302368
F1:        0.5896433591842651
 1 |  5 >>> 1.00
 1 |  3 >>> 1.00
 1 |  5 >>> 1.00
 1 |  0 >>> 1.00
 1 |  4 >>> 1.00
 1 |  2 >>> 1.00
 1 |  2 >>> 1.00
 1 |  5 >>> 1.00
 1 |  2 >>> 1.00
 1 |  1 >>> 1.00
 1 |  5 >>> 1.00
 1 |  1 >>> 1.00
 1 |  1 >>> 1.00
 1 |  5 >>> 1.00
 1 |  5 >>> 1.00
 1 |  1 >>> 1.00
 1 |  4 >>> 1.00
 1 |  4 >>> 1.00
 1 |  0 >>> 1.00
 1 |  1 >>> 1.00
 1 |  2 >>> 1.00
 1 |  1 >>> 1.00
 1 |  1 >>> 1.00
 1 |  2 >>> 1.00
 1 |  2 >>> 1.00
 1 |  5 >>> 1.00
 1 |  1 >>> 1.00
 1 |  1 >>> 1.00
 1 |  4 >>> 1.00
 1 |  5 >>> 1.00
 1 |  1 >>> 1.00
 1 |  5 >>> 1.00


In [22]:
torch.set_float32_matmul_precision('medium')
trainer.fit(model, train_dataloaders=train, val_dataloaders=test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                | Params
------------------------------------------------------
0 | layers        | Sequential          | 343   
1 | loss_function | CrossEntropyLoss    | 0     
2 | precision     | MulticlassPrecision | 0     
3 | recall        | MulticlassRecall    | 0     
4 | F1            | MulticlassF1Score   | 0     
------------------------------------------------------
343       Trainable params
0         Non-trainable params
343       Total params
0.001     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/ubuntu/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [25]:
for batch in test:
    X, y = batch
    X = X.cuda(0)
    y = y.cuda(0)
    print(f"X: {X.device}")
    print(f"y: {y.device}")
    print(f"model: {model.device}")

    model = model.cuda(0)
    output = model(X)
    # print out put device
    print(f"output: {X.device}")
    model.evaluation(output, y, model.loss_function(output, y))

    for i in range(32):
        print(f"{torch.argmax(output[i]):2d} | {torch.argmax(y[i]):2d} >>> {torch.max(output[i]):.2f}\n\t{output[i]}")
    break

X: cuda:0
y: cuda:0
model: cuda:0
output: cuda:0
CE:        1.8459675312042236
PRECISION: 0.5731707215309143
RECALL:    0.6171875
F1:        0.5759648084640503
 1 |  5 >>> 1.00
	tensor([0., 1., 1., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward0>)
 1 |  1 >>> 1.00
	tensor([0., 1., 1., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward0>)
 1 |  2 >>> 1.00
	tensor([0., 1., 1., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward0>)
 1 |  1 >>> 1.00
	tensor([0., 1., 1., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward0>)
 1 |  2 >>> 1.00
	tensor([0., 1., 1., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward0>)
 1 |  1 >>> 1.00
	tensor([0., 1., 1., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward0>)
 1 |  0 >>> 1.00
	tensor([0., 1., 1., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward0>)
 1 |  1 >>> 1.00
	tensor([0., 1., 1., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBack

In [31]:
# create 'models' directory if nonexistent
# import os
# if not os.path.exists("models"):
#     os.makedirs("models")
# # save model weights
# torch.save(model.state_dict(), "./models/model_weights.pth")